In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import csv
import os

In [30]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

ModuleNotFoundError: No module named 'imblearn'

In [31]:
ls

 Volume in drive C is Local Disk
 Volume Serial Number is 8085-52B5

 Directory of C:\Users\Robbe Verhofste\Desktop\BerkelyBootCamp\PROJECT\Energy_Analysis\API Charger Stations

05/02/2021  02:14 PM    <DIR>          .
05/02/2021  02:14 PM    <DIR>          ..
05/02/2021  09:14 AM             1,842 .gitignore
05/02/2021  01:56 PM    <DIR>          .ipynb_checkpoints
05/02/2021  02:14 PM           928,528 California_IncomeByZipDemographics.csv
05/02/2021  02:14 PM         1,022,282 Chargers_in_CA.csv
05/02/2021  09:14 AM           549,686 Chargers_in_CA.xlsx
05/02/2021  09:14 AM             7,244 chargers_per_county.xlsx
05/02/2021  09:14 AM            27,579 chargers_per_zip.xlsx
05/02/2021  09:17 AM           207,109 Charging Station API.ipynb
05/02/2021  09:14 AM            96,727 charging stations by county.png
05/02/2021  09:14 AM           159,289 Electric Charging Stations Per State.png
05/02/2021  12:01 PM    <DIR>          leaflet-csv-template
05/02/2021  02:14 PM            32

In [37]:
df_ev = pd.read_csv("Chargers_in_CA.csv",encoding='latin-1')
df_ev

,city,state,zip,access_code,access_days_time,latitude,longitude,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num,county
0,Los Angeles,CA,90015.0,public,24 hours daily - pay lot,34.040539,-118.271387,NaN,12.0,NaN,Los Angeles County
1,El Monte,CA,91731.0,public,24 hours daily,34.068720,-118.064000,NaN,3.0,NaN,Los Angeles County
2,La Jolla,CA,92037.0,public,24 hours daily,32.899470,-117.243000,NaN,1.0,NaN,San Diego County
3,Escondido,CA,92027.0,public,24 hours daily,33.098589,-117.004433,NaN,2.0,NaN,San Diego County
4,Pasadena,CA,91103.0,public,24 hours daily - pay lot,34.147620,-118.147111,NaN,16.0,NaN,Los Angeles County
...,...,...,...,...,...,...,...,...,...,...,...
11405,Williams,CA,95987.0,public,24 hours daily,39.155571,-122.150232,NaN,2.0,2.0,Colusa County
11406,Rancho Cucamonga,CA,91739.0,public,24 hours daily,34.112867,-117.533423,NaN,NaN,1.0,San Bernardino County
11407,Baker,CA,92309.0,public,24 hours daily,35.266480,-116.072764,NaN,NaN,6.0,San Bernardino County
11408,Los Angeles,CA,90012.0,public,24 hours daily,34.051235,-118.240515,NaN,NaN,1.0,Los Angeles County


In [40]:
total = df_ev['zip'].value_counts()
total


94025.0    256
95054.0    249
92618.0    209
95814.0    139
94080.0    135
          ... 
94564.0      1
91377.0      1
93426.0      1
95485.0      1
94134.0      1
Name: zip, Length: 1063, dtype: int64

In [46]:
total.to_csv('zip_counts.csv')
#I manually created the headers in the CSV, then re-brough the CSV in to join with the demographics dataset

In [43]:
total.columns = ['zip', 'count']
total

94025.0    256
95054.0    249
92618.0    209
95814.0    139
94080.0    135
          ... 
94564.0      1
91377.0      1
93426.0      1
95485.0      1
94134.0      1
Name: zip, Length: 1063, dtype: int64

In [47]:
df_zip = pd.read_csv("California_IncomeByZipDemographics.csv")
df_zip

,zip,geoid,state_name,county_name,city_name,aland,median_household_income,median_household_income_moe,average_household_income,average_household_income_moe,...,unemployment_pct,housing_units,median_value_of_owner_occupied_units,households,hh_families,hh_mc_families,hh_mc_with_own_children_under_18,hh_sp_families,hh_sp_with_own_children_under_18,hh_non_families
0,90001,86000US90001,California,Los Angeles,Florence-Graham; Huntington Park; Los Angeles,"9,071,081","$43,360","$2,437","$54,325","$1,926",...,8.7%,"14,174","$359,000","13,669","11,206","6,048","3,287","5,158","2,746","2,463"
1,90002,86000US90002,California,Los Angeles,Florence-Graham; Los Angeles; Lynwood,"7,930,602","$37,285","$1,920","$49,724","$2,082",...,9.7%,"13,546","$345,900","12,917","10,266","5,034","2,639","5,232","3,011","2,651"
2,90003,86000US90003,California,Los Angeles,Los Angeles,"9,197,635","$40,598","$1,743","$51,181","$1,916",...,9.0%,"18,523","$362,800","17,484","14,183","6,699","3,843","7,484","3,996","3,301"
3,90004,86000US90004,California,Los Angeles,Los Angeles,"7,894,528","$49,675","$1,784","$84,325","$4,154",...,4.6%,"25,192","$1,063,200","22,004","12,337","7,948","3,607","4,389","1,785","9,667"
4,90005,86000US90005,California,Los Angeles,Los Angeles,"2,807,558","$38,491","$2,241","$54,841","$2,731",...,5.6%,"18,291","$777,100","16,781","8,169","4,674","1,814","3,495","1,421","8,612"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,96148,86000US96148,California,Placer,Tahoe Vista,"3,521,555","$73,611","$26,157","$73,774","$23,366",...,0.0%,635,"$750,000",214,98,98,56,0,0,116
1760,96150,86000US96150,California,El Dorado,South Lake Tahoe,"319,971,335","$57,396","$4,821","$82,312","$4,939",...,6.7%,"23,478","$442,300","11,767","6,551","5,062","1,747","1,489",743,"5,216"
1761,96155,86000US96155,California,El Dorado,NaN,"95,941,529",-$1,-$1,-$1,-$1,...,0.0%,92,-$1,0,0,0,0,0,0,0
1762,96161,86000US96161,California,Nevada; Placer,Kingvale; Truckee,"470,961,610","$98,924","$9,536","$139,731","$13,017",...,2.0%,"17,733","$604,100","6,927","5,043","4,165","1,701",878,614,"1,884"


In [55]:
counts = pd.read_csv("zip_counts.csv",encoding='latin-1')

merge = pd.merge(counts,df_zip, on ='zip',how='outer')

merge.head(100)

merge.to_csv('zip_chargercount_demographics.csv')

AttributeError: 'DataFrame' object has no attribute 'dropnull'